In [16]:
import numpy as np 
import pandas as pd 


In [17]:
dataset = pd.read_csv('/Users/prabhnoorsingh/Documents/432AE.csv')
dataset

,Longitude,Latitude,Speed,NW,RSRP,RSRQ,NODEHEX
0,-8.424987,51.918165,44,2 LTE,-106.0,-10.0,NaN
1,-8.424987,51.918165,44,2 LTE,-106.0,-10.0,NaN
2,-8.424987,51.918165,44,2 LTE,-110.0,-12.0,NaN
3,-8.424987,51.918165,44,2 LTE,-110.0,-12.0,NaN
4,-8.424987,51.918165,44,2 LTE,-107.0,-12.0,NaN
...,...,...,...,...,...,...,...
19333,NaN,NaN,,NaN,NaN,NaN,NaN
19334,NaN,NaN,,NaN,NaN,NaN,NaN
19335,NaN,NaN,,NaN,NaN,NaN,NaN
19336,NaN,NaN,,NaN,NaN,NaN,NaN


In [18]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19338 entries, 0 to 19337
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Longitude  2201 non-null   float64
 1   Latitude   2201 non-null   float64
 2   Speed      2304 non-null   object 
 3   NW         2201 non-null   object 
 4   RSRP       2201 non-null   float64
 5   RSRQ       2201 non-null   float64
 6   NODEHEX    0 non-null      float64
dtypes: float64(5), object(2)
memory usage: 1.0+ MB


In [19]:
dataset.isna()

,Longitude,Latitude,Speed,NW,RSRP,RSRQ,NODEHEX
0,False,False,False,False,False,False,True
1,False,False,False,False,False,False,True
2,False,False,False,False,False,False,True
3,False,False,False,False,False,False,True
4,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...
19333,True,True,False,True,True,True,True
19334,True,True,False,True,True,True,True
19335,True,True,False,True,True,True,True
19336,True,True,False,True,True,True,True


In [20]:
dataset.isna().sum()

Longitude    17137
Latitude     17137
Speed        17034
NW           17137
RSRP         17137
RSRQ         17137
NODEHEX      19338
dtype: int64

In [21]:
ds1 = dataset.dropna()
ds1

,Longitude,Latitude,Speed,NW,RSRP,RSRQ,NODEHEX


In [22]:
dataset.fillna(dataset.mean(), inplace=True)

/var/folders/vv/cd5n2cw11xb4kzvgyhjlwj680000gn/T/ipykernel_86104/2429533873.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  dataset.fillna(dataset.mean(), inplace=True)


In [24]:
# separate dataset 
X = dataset.iloc[:,[0,1,2,4,5,6]]
Y = dataset.iloc[:,[3]]

In [25]:
from sklearn.preprocessing import LabelEncoder


In [26]:
lEncoder = LabelEncoder()

In [27]:
from sklearn.model_selection import train_test_split 

X_train, Y_train, X_test, Y_test = train_test_split(X, Y, test_size=0.25, random_state=0)

In [28]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()



In [29]:
X_train = sc_x.fit_transform(X_train)
X_test = sc_x.fit_transform(X_test)

ValueError: could not convert string to float: ' '

In [30]:
print(X_test)

        NW
7479   NaN
2610   NaN
16211  NaN
11920  NaN
8634   NaN
...    ...
9225   NaN
13123  NaN
9845   NaN
10799  NaN
2732   NaN

[14503 rows x 1 columns]


In [31]:
#the random forest classifier

In [43]:
from sklearn.model_selection import train_test_split

SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.2, 
                                                    random_state=SEED)

In [44]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=3, 
                             max_depth=2,
                             random_state=SEED)

In [45]:
# Fit RandomForestClassifier
rfc.fit(X_train, y_train)
# Predict the test set labels
y_pred = rfc.predict(X_test)

ValueError: could not convert string to float: ' '

In [46]:
# Import `tree` module
from sklearn import tree

features = X.columns.values # The name of each column
classes = ['0', '1', '2'] # The name of each class
# You can also use low, medium and high risks in the same order instead
# classes = ['low risk', 'medium risk', 'high risk']

for estimator in rfc.estimators_:
    print(estimator)
    plt.figure(figsize=(12,6))
    tree.plot_tree(estimator,
                   feature_names=features,
                   class_names=classes,
                   fontsize=8, 
                   filled=True, 
                   rounded=True)
    plt.show()

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

In [47]:
from sklearn.metrics import classification_report, confusion_matrix

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d').set_title('Maternal risks confusion matrix (0 = low risk, 1 = medium risk, 2 = high risk)')

print(classification_report(y_test,y_pred))

NameError: name 'y_pred' is not defined

In [48]:
# Organizing feature names and importances in a DataFrame
features_df = pd.DataFrame({'features': rfc.feature_names_in_, 'importances': rfc.feature_importances_ })

# Sorting data from highest to lowest
features_df_sorted = features_df.sort_values(by='importances', ascending=False)

# Barplot of the result without borders and axis lines
g = sns.barplot(data=features_df_sorted, x='importances', y ='features', palette="rocket")
sns.despine(bottom = True, left = True)
g.set_title('Feature importances')
g.set(xlabel=None)
g.set(ylabel=None)
g.set(xticks=[])
for value in g.containers:
    g.bar_label(value, padding=2)

AttributeError: 'RandomForestClassifier' object has no attribute 'estimators_'

In [49]:
rfc_ = RandomForestClassifier(n_estimators=900, 
                             max_depth=7,
                             random_state=SEED)
rfc_.fit(X_train, y_train)
y_pred = rfc_.predict(X_test)

ValueError: could not convert string to float: ' '

In [50]:
cm_ = confusion_matrix(y_test, y_pred)
sns.heatmap(cm_, annot=True, fmt='d').set_title('Maternal risks confusion matrix (0 = low risk, 1 = medium risk, 2 = high risk) for 900 trees with 8 levels')

print(classification_report(y_test,y_pred))

NameError: name 'y_pred' is not defined

In [51]:
#hyperparameters
hyperparameters = {'n_estimators': [10, 50, 100, 200],
                   'max_depth': [5, 10, 20, 50]}

In [52]:
rf = RandomForestClassifier()

In [53]:
clf = GridSearchCV(rf, hyperparameters, cv=5)

NameError: name 'GridSearchCV' is not defined